## 향미방

### 1> 데이터 확인

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as spst

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# raw_data = pd.read_excel('./data/raw_data/(Metabolite)GC-Volatile_250529.xlsx', sheet_name='RAW_DATA', index_col=0).T
# raw_data.head(2)

Compounds,Carbon disulfide,Methylsulfanylmethane (Dimethyl sulfide),2-Methylheptane,Furan,Propan-2-one,4-Methyloctane,(E)-oct-2-ene,2-Methylfuran,Butan-2-one,Nonane,...,Undecan-4-ylbenzene (4-Phenylundecane),Phenylmethanol,"(1-hydroxy-2,4,4-trimethylpentan-3-yl) 2-methylpropanoate",Methylsulfonylmethane (Dimethyl sulfone),Dodecan-3-ylbenzene (3-Phenyldodecane),2-Phenylethanol,2-Phenylacetonitrile,"1,4-Butanediol",Benzeneacetonitrile,Nonadecane
0_1,0.130670,0.0,0.000000,0.0,1.435842,0.517948,0.0,0.0,0.175037,0.0,...,0.0,0.0,0.0,0.324772,0.0,0.000000,0.03144,0.0,0.0,0.0
0_2,0.167196,0.0,0.027512,0.0,0.952583,0.000000,0.0,0.0,0.332735,0.0,...,0.0,0.0,0.0,0.004501,0.0,0.005626,0.00000,0.0,0.0,0.0


In [2]:
data = pd.read_excel('./data/GC_Volatile_data.xlsx', index_col=0)
data.head(2)

,Carbon disulfide,Methylsulfanylmethane (Dimethyl sulfide),2-Methylheptane,Furan,Propan-2-one,4-Methyloctane,(E)-oct-2-ene,2-Methylfuran,Butan-2-one,Nonane,...,Undecan-4-ylbenzene (4-Phenylundecane),Phenylmethanol,"(1-hydroxy-2,4,4-trimethylpentan-3-yl) 2-methylpropanoate",Methylsulfonylmethane (Dimethyl sulfone),Dodecan-3-ylbenzene (3-Phenyldodecane),2-Phenylethanol,2-Phenylacetonitrile,"1,4-Butanediol",Benzeneacetonitrile,Nonadecane
A0-1,0.130670,0.0,0.000000,0.0,1.435842,0.517948,0.0,0.0,0.175037,0.0,...,0.0,0.0,0.0,0.324772,0.0,0.000000,0.03144,0.0,0.0,0.0
A0-2,0.167196,0.0,0.027512,0.0,0.952583,0.000000,0.0,0.0,0.332735,0.0,...,0.0,0.0,0.0,0.004501,0.0,0.005626,0.00000,0.0,0.0,0.0


#### 1-1> 데이터 확인

In [5]:
print(raw_data.shape)
print(raw_data.columns)

(270, 191)
Index(['Carbon disulfide', 'Methylsulfanylmethane (Dimethyl sulfide) ',
       '2-Methylheptane', 'Furan', 'Propan-2-one', '4-Methyloctane  ',
       '(E)-oct-2-ene', '2-Methylfuran', 'Butan-2-one  ', 'Nonane',
       ...
       'Undecan-4-ylbenzene (4-Phenylundecane)', 'Phenylmethanol',
       '(1-hydroxy-2,4,4-trimethylpentan-3-yl) 2-methylpropanoate  ',
       'Methylsulfonylmethane (Dimethyl sulfone)',
       'Dodecan-3-ylbenzene (3-Phenyldodecane) ', '2-Phenylethanol  ',
       '2-Phenylacetonitrile', '1,4-Butanediol', 'Benzeneacetonitrile',
       'Nonadecane'],
      dtype='object', name='Compounds', length=191)


In [8]:
# 0 값의 개수와 비율
zcnt = 0
total_cnt = len(raw_data.columns) * len(raw_data.index)
for c in raw_data.columns:
    zcnt += len(raw_data.loc[raw_data[c] == 0])

print(zcnt, np.round(zcnt/total_cnt, 4))

39144 0.759


In [9]:
# null 값의 개수와 비율
nullcnt = raw_data.isna().sum().sum()
print(nullcnt, np.round(nullcnt / total_cnt, 4))

3 0.0001


In [10]:
# 0이 아닌 값의 개수와 비율
nzcnt = 0
for c in raw_data.columns:
    nzcnt += len(raw_data.loc[raw_data[c] != 0])

print(nzcnt, np.round(nzcnt/total_cnt, 4))

12426 0.241


#### 1-2> 데이터 수정 및 분할

* 데이터 인덱스 수정 (0_1 -> A0-1)
* 컬럼 수정
* sheet 분류

In [11]:
raw_data.index = [('A' + str(x) + '-' + str(y)) for x in [0, 1, 3, 5, 7, 9, 11, 14, 18] for y in range(1, 31)]
raw_data.columns = raw_data.columns.str.strip()
raw_data.head(2)

Compounds,Carbon disulfide,Methylsulfanylmethane (Dimethyl sulfide),2-Methylheptane,Furan,Propan-2-one,4-Methyloctane,(E)-oct-2-ene,2-Methylfuran,Butan-2-one,Nonane,...,Undecan-4-ylbenzene (4-Phenylundecane),Phenylmethanol,"(1-hydroxy-2,4,4-trimethylpentan-3-yl) 2-methylpropanoate",Methylsulfonylmethane (Dimethyl sulfone),Dodecan-3-ylbenzene (3-Phenyldodecane),2-Phenylethanol,2-Phenylacetonitrile,"1,4-Butanediol",Benzeneacetonitrile,Nonadecane
A0-1,0.130670,0.0,0.000000,0.0,1.435842,0.517948,0.0,0.0,0.175037,0.0,...,0.0,0.0,0.0,0.324772,0.0,0.000000,0.03144,0.0,0.0,0.0
A0-2,0.167196,0.0,0.027512,0.0,0.952583,0.000000,0.0,0.0,0.332735,0.0,...,0.0,0.0,0.0,0.004501,0.0,0.005626,0.00000,0.0,0.0,0.0


In [ ]:
# raw_data.to_excel('./data/GC_Volatile_data.xlsx', sheet_name='RAW_DATA')

In [12]:
raw_sheet2 = pd.read_excel('./data/raw_data/(Metabolite)GC-Volatile_250529.xlsx', sheet_name='RAW_DATA (화합물분류)')
raw_sheet2 = raw_sheet2.drop(columns=raw_sheet2.columns[2:], axis=1)
raw_sheet2.head(2)

,Compounds class,Compounds
0,S containing compounds,Carbon disulfide
1,S containing compounds,Methylsulfanylmethane (Dimethyl sulfide)


In [13]:
raw_sheet2 = raw_sheet2[['Compounds', 'Compounds class']]
raw_sheet2.columns = ['Compounds', 'Class']
raw_sheet2['Compounds'] = raw_sheet2['Compounds'].str.strip()
raw_sheet2.head(2)

,Compounds,Class
0,Carbon disulfide,S containing compounds
1,Methylsulfanylmethane (Dimethyl sulfide),S containing compounds


In [ ]:
# raw_sheet2.to_excel('./data/GC_Volatile_class.xlsx', sheet_name='RAW_DATA (class)',index=0)

#### 1-3> 일자별 분할

In [3]:
data['days'] = data.index.str.split('A').str[1]
data['days'] = data['days'].str.split('-').str[0].astype(int)
data['days']

A0-1       0
A0-2       0
A0-3       0
A0-4       0
A0-5       0
          ..
A18-26    18
A18-27    18
A18-28    18
A18-29    18
A18-30    18
Name: days, Length: 270, dtype: int32

In [15]:
data['days'].unique()

array([ 0,  1,  3,  5,  7,  9, 11, 14, 18])

In [17]:
for i in data['days'].unique():
    data_day = data.loc[data['days'] == i]
    data_day = data_day.drop(columns='days', axis=1)
    path = './data/GC_Volatile_day' + f"{i:02d}" + '.xlsx'
    data_day.to_excel(path, sheet_name=f"day{i:02d}", index=0)

In [20]:
vol00 = pd.read_excel('./data/GC_Volatile_day00.xlsx')
vol01 = pd.read_excel('./data/GC_Volatile_day01.xlsx')
vol03 = pd.read_excel('./data/GC_Volatile_day03.xlsx')

vol05 = pd.read_excel('./data/GC_Volatile_day05.xlsx')
vol07 = pd.read_excel('./data/GC_Volatile_day07.xlsx')
vol09 = pd.read_excel('./data/GC_Volatile_day09.xlsx')

vol11 = pd.read_excel('./data/GC_Volatile_day11.xlsx')
vol14 = pd.read_excel('./data/GC_Volatile_day14.xlsx')
vol18 = pd.read_excel('./data/GC_Volatile_day18.xlsx')

vol18.head(2)

,Carbon disulfide,Methylsulfanylmethane (Dimethyl sulfide),2-Methylheptane,Furan,Propan-2-one,4-Methyloctane,(E)-oct-2-ene,2-Methylfuran,Butan-2-one,Nonane,...,Undecan-4-ylbenzene (4-Phenylundecane),Phenylmethanol,"(1-hydroxy-2,4,4-trimethylpentan-3-yl) 2-methylpropanoate",Methylsulfonylmethane (Dimethyl sulfone),Dodecan-3-ylbenzene (3-Phenyldodecane),2-Phenylethanol,2-Phenylacetonitrile,"1,4-Butanediol",Benzeneacetonitrile,Nonadecane
0,0.0,0.059838,0.0,0,0.000000,0.000000,0.0,0,0.446364,0,...,0,0,0,0.042213,0,0.309898,0,0,0.0,0
1,0.0,0.242405,0.0,0,0.442002,0.905596,0.0,0,0.525117,0,...,0,0,0,0.034399,0,0.108116,0,0,0.0,0


In [26]:
from scipy.stats import f_oneway
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [27]:
anova1 = data[['3-Methylbutan-1-ol', 'days']]
anova1.columns = ['Methyl', 'days']
anova1.head(2)

,Methyl,days
A0-1,0.0,0
A0-2,0.0,0


In [ ]:
grouped = anova1.groupby('days')['Methyl'].apply(list)
grouped

days
0     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.060348146, 0....
5     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
7     [0.0, 0.0, 0.0, 0.0, 0.0, 0.018619532, 0.0, 0....
9     [0.134831896, 0.143631984, 0.007450522, 0.0113...
11    [5.980286176, 16.1260892, 0.616330194, 5.96112...
14    [6.946721674, 6.542003421, 2.779050382, 1.8441...
18    [6.090099166, 5.796483916, 3.570555808, 4.5594...
Name: 3-Methylbutan-1-ol, dtype: object

In [34]:
f_stat, p_value = f_oneway(*grouped)
print("f_stat: ", f_stat)
print("p_value: ", p_value)

f_stat:  62.952772136061434
p_value:  1.3531844442140145e-56


In [38]:
model = ols("Methyl ~ C(days)", data=anova1).fit()
anova_table = anova_lm(model)

print(anova_table)

             df       sum_sq     mean_sq          F        PR(>F)
C(days)     8.0  1935.106392  241.888299  62.952772  1.353184e-56
Residual  261.0  1002.860460    3.842377        NaN           NaN


#### 사후 검정

ANOVA 검정은 3개 이상의 집단이 동일한지를 비교분석하는 통계 방법  
하지만 검정을 통해 집단 간 차이의 유무만 확인 가능할 뿐, 어떤 집단간 차이가 있는지는 알 수 없다.  
P-value > 0.5 : A = B = C  
P-value < 0.5 : A!=B or B!=C or C!=A or A!=B!=C  

* 사후 분석을 통해 각 집단별로 어떤 차이가 있는지 확인 가능  

tukey : 비교 대상 표본 수가 동일한 경우 가장 많이 사용되는 사후검정 기법  

* 대사체와 날짜간 사후 검정을 통해 일별로 대사체가 차이가 있는지 확인 가능

In [ ]:
from statsmodels.sandbox.stats.multicomp import MultiComparison

In [80]:
comp = MultiComparison(anova1['Methyl'], anova1['days'])
result = comp.tukeyhsd()
tukey_df = pd.DataFrame(data=result._results_table.data[1:], columns=result._results_table.data[0])
tukey_df.loc[tukey_df['reject'] == True]

,group1,group2,meandiff,p-adj,lower,upper,reject
5,0,11,7.0538,0.0000,5.4706,8.6369,True
6,0,14,4.7921,0.0000,3.2089,6.3753,True
7,0,18,4.7733,0.0000,3.1901,6.3564,True
12,1,11,7.0538,0.0000,5.4706,8.6369,True
13,1,14,4.7921,0.0000,3.2089,6.3753,True
14,1,18,4.7733,0.0000,3.1901,6.3564,True
18,3,11,7.0421,0.0000,5.4589,8.6253,True
19,3,14,4.7804,0.0000,3.1973,6.3636,True
20,3,18,4.7616,0.0000,3.1784,6.3447,True
23,5,11,7.0538,0.0000,5.4706,8.6369,True


In [90]:
s = ''
for idx, row in tukey_df.loc[tukey_df['reject'] == True].iterrows():
    s += (str(row['group1']) + '-' + str(row['group2']) + ".")

s = s.split('.')
for i in range(len(s)):
    print(s[i], end='\t')
    if i > 0 and i % 5 == 0:
        print()

0-11	0-14	0-18	1-11	1-14	1-18	
3-11	3-14	3-18	5-11	5-14	
5-18	7-11	7-14	7-18	9-11	
9-14	9-18	11-14	11-18		


In [ ]:
0-3     0-9     0-11    0-14    0-18
1-3     1-9     1-11    1-14    1-18
3-5     3-9     3-11    3-14    3-18
5-9     5-11    5-14    5-18
7-9     7-11    7-14    7-18
9-11    9-14    9-18

### 3> 

In [ ]:
df2_mean_std = []
days = df2['days'].unique()
for day in df2['day'].unique():
    tempdf = pd.DataFrame()
    for col in df2.columns:
        if col == 'day':
            continue
        ser = df2.loc[df2['day'] == day, col]
        ddf = pd.DataFrame({'mean': [ser.mean()], 'std': [ser.std()]}, index=[col])
        tempdf = pd.concat([tempdf, ddf])

    df2_mean_std.append(tempdf)

display(df2_mean_std[0])
display(df2_mean_std[8])

In [ ]:
def VM_meanstd(target, df=df2_mean_std, days=days, draw_errorbar=True):
    vm_df = pd.DataFrame()
    for i in range(len(df)):
        tdf = df[i].loc[[target]]
        tdf.index = days[i:i+1]
        vm_df = pd.concat([vm_df, tdf.iloc[0:1, :]])
    
    if draw_errorbar:
        plt.errorbar(
            vm_df.index, vm_df['mean'], yerr=vm_df['std'],
            fmt='o', linestyle='-',
            color='dodgerblue', ecolor='gray'
        )
        plt.title("Changes of " + target + "(Mean, Std)")
        plt.xlabel('days')
        plt.ylabel('Mean')
        plt.xticks(days)
        # plt.yticks(np.arange(-0.05, 0.26, 0.05))
        plt.grid(axis='y')
        plt.show()

    return vm_df

In [ ]:
three_hydroxybutan_2_one_Acetoin_df = VM_meanstd('3-hydroxybutan-2-one (Acetoin)')

In [ ]:
def eda_single_numeric(var, df, describe=True):
    if describe:
        display(df[var].describe().to_frame().T)
    
    plt.subplot(2,1,1)
    sns.histplot(df[var])
    
    plt.subplot(2,1,2)
    sns.boxplot(x=var, data=df)
    plt.grid()
    plt.tight_layout()
    plt.show()

### PLS-DA

In [25]:
data.shape[0]

270

In [28]:
tdata = data.copy() # 0이 90% 이상인 컬럼 제거
tdata = tdata.replace(0, np.NaN)
drop_cols = []
for c in tdata.columns:
    x = tdata[c].isna().sum()
    zratio = x / data.shape[0] # 0 개수 / 270
    
    if zratio >= 0.9:
        drop_cols.append(c)

print(len(drop_cols))
print(drop_cols[:5])

106
['(E)-oct-2-ene', '2-Methylfuran', 'Nonane', '2,5-Dimethylfuran', 'Cyclopentanol']


In [31]:
tdata2 = tdata.drop(columns=drop_cols, axis=1)
tdata2 = tdata2.replace(np.NaN, 0)
tdata2.shape

(270, 86)

In [33]:
target = 'days'

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [57]:
X = tdata2.drop(columns=target, axis=1)
Y = tdata2.loc[:, target].astype(int)
Y = pd.get_dummies(Y, drop_first=True, dtype=int)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

def my_PLS_DA(X_train, X_test, Y_train, Y_test, n_components_=2):
    pls = PLSRegression(n_components=n_components_)
    pls.fit(X_train, Y_train)

    y_pred = pls.predict(X_test)
    y_pred_flatten = [x.argmax() for x in y_pred]

    
    return pls, y_pred_flatten

pls_model, y_pred = my_PLS_DA(x_train, x_test, y_train, y_test)

In [63]:
# print(accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
# print(classification_report(y_test, y_pred))

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets